# Connecting with a CRM

In [5]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources .
# It does this by quickly running through all the code from the previous notebook.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Found: mugs-customer-support-agent
Deleting alias: MyAgentAlias (ID: APZAH2Z3WR)
Deletion initiated for alias: MyAgentAlias
Alias MyAgentAlias has been successfully deleted.
Deleting alias: AgentTestAlias (ID: TSTALIASID)
Deletion initiated for alias: AgentTestAlias
Alias AgentTestAlias has been successfully deleted.
Deleting agent: mugs-customer-support-agent (ID: PFJUQGUVC7)
Deletion initiated for agent: mugs-customer-support-agent
Waiting for agent mugs-customer-support-agent to be deleted...
Agent mugs-customer-support-agent has been successfully deleted.
Agent reset process completed.
Deleting Lambda function: dlai-support-agent-LK6RJ
Lambda function dlai-support-agent-LK6RJ deleted successfully.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting fo

In [10]:
import boto3
import uuid
from helper import *

In [11]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [12]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: To get a refund for your broken mug, I will need to transfer you to a
       human customer support agent who can assist you further. They
       will be able to access your account details and initiate the
       refund process. Please hold while I transfer you.

Session ID: b02fc760-0f88-445f-b24b-6fb6d54d5808


In [13]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [14]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

ConflictException: An error occurred (ConflictException) when calling the CreateAgentActionGroup operation: Could not perform Create operation, since the customer-support-actions (id: J1PSW7LLGW) with the same name customer-support-actions already exists

In [ ]:
create_agent_action_group_response

In [ ]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [ ]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

In [ ]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

In [ ]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

### Now use the agent with functions

In [ ]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [15]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: I apologize, but I do not have the ability to directly process refunds
       for broken items. To get a refund for your broken mug, I will
       need to transfer you to a human customer support agent who can
       access your account details and initiate the refund process.
       Please hold while I transfer you to an agent who can assist you
       further.

Session ID: b02fc760-0f88-445f-b24b-6fb6d54d5808


In [16]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  Okay, let me try to assist you with getting a refund for your broken
  mug.  First, I'll need to gather some more details about your
  purchase and the issue with the mug:

Agent's thought process:
  It seems I do not have access to the necessary customer details or
  refund process to handle this request directly. The best I can do is
  route this to a human customer support agent who can properly assist
  Mike with getting a refund for his broken mug.

Observation:
  Type: FINISH

Final response:
  I apologize, but I do not have the ability to directly process
  refunds for broken items. To get a refund for your broken mug, I
  will need to transfer you to a human customer support agent who can
  access your account details and initiate the refund process. Please
  hold while I transfer you to an agent who can assist you further.


Session ID: b02fc760-0f88-445f-b24b-6fb6d54d5808
